# Trabajo Práctico N°3


    img_align_celeba.zip: All the face images, cropped and aligned
    list_eval_partition.csv: Recommended partitioning of images into training, validation, testing sets. Images 1-162770 are training, 162771-182637 are validation, 182638-202599 are testing
    list_bbox_celeba.csv: Bounding box information for each image. "x_1" and "y_1" represent the upper left point coordinate of bounding box. "width" and "height" represent the width and height of bounding box
    list_landmarks_align_celeba.csv: Image landmarks and their respective coordinates. There are 5 landmarks: left eye, right eye, nose, left mouth, right mouth
    list_attr_celeba.csv: Attribute labels for each image. There are 40 attributes. "1" represents positive while "-1" represents negative


In [ ]:
# un poco menos de warnings de tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# de python, para especificar rutas de archivos y directorios
from pathlib import Path

# lib para trabajar con arrays
import numpy as np
import pandas as pd

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
#Imagenes_DIR = Path('./dataset_TP3/img_align_celeba')
IMG_PATH = Path('/home/matias/Documentos/archive/img_align_celeba/img_align_celeba')
atributos_DIR = Path('/home/matias/Documentos/archive/list_attr_celeba.csv')
set_DIR = Path('/home/matias/Documentos/archive/list_eval_partition.csv')

In [ ]:
df = pd.read_csv(atributos_DIR, usecols=['image_id','No_Beard'])

# Reset the columns values to categorical./
df.loc[df['No_Beard'] == -1,'No_Beard'] = "Barba"
df.loc[df['No_Beard'] == 1,'No_Beard'] = "No_Barba"

df.head(20)

In [ ]:
"""
dfp = pd.read_csv(set_DIR)
#dfp.shape(20)
particion=np.array(dfp)
atributos=np.array(df)
x=len(df)

def dividirDataset(atributos,particion):
    iv=0
    it=0
    ite=0
    for i in range(202599):
        if (particion[i][1]==0):
                train_df[it]=atributos[i][1]
                it=it+1
            else:
                if (particion[i][1]==1):
                    validation_df[iv]=atributos[i][1]
                    iv=iv+1
                else:
                    test_df[ite]=atributos[i][1]
                    ite=ite+1
                    
#dividirDataset(atributos,particion)
"""

In [ ]:
category_count = df["No_Beard"].value_counts()
print(category_count)

higher_category = list(category_count.index)[0]

In [ ]:
df["No_Beard"].value_counts().plot.bar()

In [ ]:
atributos=np.array(df)
atributos[148010][1] # uno menos 

#hacer labels con lo de los atributos.no_beard
#hacer x_train(imagenes, tamaño, tamaño)
#hacer y_train(atributos.no_beard)

In [ ]:
#images_df = image_datagen.flow_from_directory(Imagenes_DIR)
#completa=completa.merge(train, on=name, how="left")
#validation = images_reader.flow_from_directory(VALIDATION_DIR, **READ_PARAMS)

train_df, test_df = train_test_split(df, train_size=0.80)
test_df, validation_df = train_test_split(test_df, test_size=0.9)
print(len(train_df))
print(len(test_df))
print(len(validation_df))
train_df.head(20)

In [ ]:
IMAGE_SIZE = (218, 178)
BATCH_SIZE = 15

train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    #width_shift_range=0.1,
    #height_shift_range=0.1
)

validation_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    #width_shift_range=0.1,
    #height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    '/home/matias/Documentos/archive/img_align_celeba/img_align_celeba'+'/', 
    x_col='image_id',
    y_col='No_Beard',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    #validate_filenames=False
)

In [ ]:
def sample_images(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = dataset.next()
    for i in range(10):
        ax = plt.subplot(6,6,i+1)
        plt.imshow(images[i])
        plt.title(labels[i])
        #print(images[17][26])
        
        #print(labels[i])
        #print(i)
        #clase=atributos[i][26]
        #plt.title(class_name[clase])
        plt.axis("off")


In [ ]:
sample_images(train_generator)

In [ ]:
base_vgg_model = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape= IMAGE_SIZE + (3,))

In [ ]:
base_vgg_model.trainable = False
base_vgg_model.summary()


In [ ]:
vgg_model = Sequential(
    [
    base_vgg_model,
    Flatten(),
    Dense(256,activation='relu'),
    Dense(256,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='sigmoid')
]
)

In [ ]:
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=4, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.001)
callbacks = [earlystop, learning_rate_reduction]

history = vgg_model.fit(train_generator, validation_data = validation_generator
                        , validation_steps=len(validation_df)//BATCH_SIZE
                        ,steps_per_epoch=len(train_df)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=callbacks)



In [ ]:
pretrained_model = VGG16(include_top=False, input_shape= IMAGE_SIZE + (3,))
pretrained_model.trainable = False

vgg16_model = Sequential(
    [
    pretrained_model,
    Convolution2D(filters=8, kernel_size=(2, 2), strides=1, activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(50,activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='sigmoid')
]
)

vgg16_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

In [ ]:
model_weights_at_epochs = {}

class OurCustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        model_weights_at_epochs[epoch] = self.model.get_weights()
        
history = vgg16_model.fit(
    train_generator, validation_data = validation_generator
                        , validation_steps=len(validation_df)//BATCH_SIZE
                        ,steps_per_epoch=len(train_df)//BATCH_SIZE,
                        epochs=10, verbose = 1, callbacks=[OurCustomCallback()]
)